- `sudo dmidecode -t baseboard`
    - 查看自己的主板型号以及 onboard device

In [1]:
from IPython.display import Image

In [3]:
# https://www.supermicro.com/zh_tw/products/motherboard/x12dpg-qt6
Image(url='https://www.supermicro.com/files_SYS/images/Motherboard/X12DPG-QT6.png', width=500)

- CPU插槽：图中有两个大块区域带有黄色警告标签，这是两个CPU插槽，用于安装服务器级的多核处理器。通常情况下，这种主板支持双路处理器。
- 内存插槽：这些蓝色和黑色的长条插槽是内存插槽（DIMM插槽）。这种主板一般支持大量的内存，以满足服务器和数据中心的高性能需求。
    - 每个CPU插槽周围有8个内存插槽。
- PCIe插槽：图中的多个黑色插槽是PCIe插槽，用于安装扩展卡，如显卡、网卡和存储控制器。
    - 1 PCIe 4.0 x8,
    - 6 PCIe 4.0 x16


## 内存条

- `sudo dmidecode -t memory`
- `sudo lshw -short -C memory`
    - 查看支持几个内存条及已经安装了几个内存条；